In [2]:
import torch
from torch.utils.data import DataLoader, TensorDataset
from modules.training import train_model

import os
from torchvision import transforms
from torchvision import datasets, transforms
from torchvision.models import resnet18
import torch.nn as nn
import torch.optim as optim
from torchsummary import summary

# Import data post-CNN and train a shallow network -> Best results

In [2]:
# Load the training data
data_train = torch.load('data_images/train.pt')
X_train = data_train['Features'][:613, :]  # shape: (N, H)
Y_train = data_train['Targets'][:613]  # shape: (N,)

# Load the validation data
data_val = torch.load('data_images/valid.pt')
X_val = data_val['Features']  # shape: (N, H)
Y_val = data_val['Targets']  # shape: (N,)

print(f"Training data shape: {X_train.shape}, {Y_train.shape}")
print(f"Validation data shape: {X_val.shape}, {Y_val.shape}")

# Create DataLoaders
batch_size = 32
train_ds = TensorDataset(X_train, Y_train)
train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=True)
val_ds = TensorDataset(X_train, Y_train)
val_loader = DataLoader(train_ds, shuffle=True)

shared_weights = nn.Linear(512, 4)


# Define a simple NN
class SimpleNN(nn.Module):
    def __init__(self, X, Y):
        super().__init__()
        self.fc1 = deepcopy(shared_weights)

        # Loss and optimizer are defined here:
        self.criterion = nn.CrossEntropyLoss()
        self.optimizer = optim.Adam(self.parameters(), lr=0.001)
        self.scheduler = optim.lr_scheduler.StepLR(self.optimizer, step_size=10, gamma=0.9)

    def forward(self, x):
        x = self.fc1(x)
        # x = self.softmax(x)
        return x

model = SimpleNN(X_train, Y_train)
print(summary(model, input_size=X_train.shape[1:]))

# train_model(model, train_loader, val_loader, epochs=10)

Training data shape: torch.Size([613, 512]), torch.Size([613])
Validation data shape: torch.Size([216, 512]), torch.Size([216])
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Linear-1                    [-1, 4]           2,052
Total params: 2,052
Trainable params: 2,052
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.00
Params size (MB): 0.01
Estimated Total Size (MB): 0.01
----------------------------------------------------------------
None


# Import images, preprocess them using resnet with fc=Identity() then train the same simple network -> Best results
Results should be the same as before and they are: at last epoch (100) we get Train Acc: 0.961 | Val Acc: 0.964

In [3]:
dataset_path = os.path.join(os.getcwd(), 'data_images')
print(dataset_path)

# Mean e std per immagini RGB normalizzate su [-1, 1]
mean = [0.5, 0.5, 0.5]
std = [0.5, 0.5, 0.5]

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean, std)
])

train_dataset = datasets.ImageFolder(root=os.path.join(dataset_path, "train"), transform=transform)
val_dataset = datasets.ImageFolder(root=os.path.join(dataset_path, "valid"), transform=transform)

train_loader = DataLoader(train_dataset, batch_size=len(train_dataset), shuffle=False)
val_loader = DataLoader(val_dataset, batch_size=len(val_dataset), shuffle=False)

model = resnet18(pretrained=True)

for param in model.parameters():
    param.requires_grad = False

model.fc = nn.Identity()

if True:  # Change to True if you want to train the model
    model.eval()
    with torch.no_grad():
        for inputs, labels in train_loader:
            print("starting")
            outputs = model(inputs)
            mapped_data_train = outputs
            labels_train = labels
        for inputs, labels in val_loader:
            outputs = model(inputs)
            mapped_data_val = outputs
            labels_val = labels

    train_ds = TensorDataset(mapped_data_train, labels_train)
    train_loader = DataLoader(train_ds, batch_size=32, shuffle=True)

    val_ds = TensorDataset(mapped_data_val, labels_val)
    val_loader = DataLoader(train_ds, shuffle=False)

    model = SimpleNN(mapped_data_train, labels_train)
    train_model(model, train_loader, val_loader, epochs=100)

/home/user/DAML-project/data_images


/home/user/DAML-project/.venv/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/user/DAML-project/.venv/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


starting
Epoch 1/100 | Train Loss: 26.492 | Train Acc: 0.380 | Val Acc: 0.542
Epoch 2/100 | Train Loss: 19.768 | Train Acc: 0.582 | Val Acc: 0.582
Epoch 3/100 | Train Loss: 17.692 | Train Acc: 0.659 | Val Acc: 0.708
Epoch 4/100 | Train Loss: 16.102 | Train Acc: 0.674 | Val Acc: 0.750
Epoch 5/100 | Train Loss: 15.416 | Train Acc: 0.716 | Val Acc: 0.744
Epoch 6/100 | Train Loss: 14.192 | Train Acc: 0.741 | Val Acc: 0.757
Epoch 7/100 | Train Loss: 13.918 | Train Acc: 0.718 | Val Acc: 0.736
Epoch 8/100 | Train Loss: 13.120 | Train Acc: 0.723 | Val Acc: 0.768
Epoch 9/100 | Train Loss: 12.383 | Train Acc: 0.767 | Val Acc: 0.768
Epoch 10/100 | Train Loss: 11.882 | Train Acc: 0.790 | Val Acc: 0.780
Epoch 11/100 | Train Loss: 11.354 | Train Acc: 0.801 | Val Acc: 0.837
Epoch 12/100 | Train Loss: 11.730 | Train Acc: 0.814 | Val Acc: 0.835
Epoch 13/100 | Train Loss: 10.715 | Train Acc: 0.822 | Val Acc: 0.811
Epoch 14/100 | Train Loss: 10.503 | Train Acc: 0.829 | Val Acc: 0.801
Epoch 15/100 | Train

# Train resnet with fc=nn.Linear() -> Bad results
Takes around 36s/epoch on my machine

In [ ]:
train_dataset = datasets.ImageFolder(root=os.path.join(dataset_path, "train"), transform=transform)
val_dataset = datasets.ImageFolder(root=os.path.join(dataset_path, "valid"), transform=transform)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=len(val_dataset), shuffle=False)

model = resnet18(pretrained=True)

model.fc = nn.Linear(512, 4)
<!-- # model.fc = SimpleNN(X_train, Y_train)

# model.eval()
# with torch.no_grad():
#     for inputs, labels in train_loader:
#         print("starting")
#         outputs = model(inputs)
#         mapped_data = outputs

# model = SimpleNN(mapped_data, labels) -->
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

model.criterion = nn.CrossEntropyLoss()
model.optimizer = optim.Adam(model.fc.parameters(), lr=0.001)
model.scheduler = optim.lr_scheduler.StepLR(model.optimizer, step_size=10, gamma=0.9)

train_model(model, train_loader, val_loader, epochs=100)

# TODO: Train resnet both ways but using batches instead of preprocessing results in one batch

# Data augmentation su determinate classi

In [1]:
#modifica ImageFolder per poter applicare DA diverse a seconda delle classi
from torchvision.datasets import ImageFolder

class BalancedAugmentDataset(ImageFolder):
    def __init__(self, root, transform_common=None, transform_augmented=None, classes_to_augment=None):
        super().__init__(root, transform=None)
        self.transform_common = transform_common
        self.transform_augmented = transform_augmented
        self.classes_to_augment = classes_to_augment or []

    def __getitem__(self, index):
        path, label = self.samples[index]
        image = self.loader(path)

        # Applica augmentation solo se la classe è tra quelle specificate
        if self.classes[label] in self.classes_to_augment:
            if self.transform_augmented:
                image = self.transform_augmented(image)
        else:
            if self.transform_common:
                image = self.transform_common(image)

        return image, label
    
# Trasformazione base (no augmentation)
base_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean, std)
])

# Augmentation solo per le classi minoritarie
augmented_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(15),
    transforms.ColorJitter(brightness=0.2, contrast=0.2),
    transforms.RandomAffine(degrees=0, translate=(0.05, 0.05)),
    transforms.ToTensor(),
    transforms.Normalize(mean, std)
])

# provo ad aumentare le classi 1 e 3
classes_to_augment = ["Adgelcarcinoma", "Noncancer"]

# carica il training set con le nuove classi
train_dataset = BalancedAugmentDataset(
    root=os.path.join(dataset_path, "train"),
    transform_common=base_transform,
    transform_augmented=augmented_transform,
    classes_to_augment=classes_to_augment
)

NameError: name 'transforms' is not defined

Ecco i risultati:
Accuracy : 0.6667
F1 Score : 0.6739
Precision: 0.7011
Recall   : 0.6647

Detailed per-class metrics:
                     precision    recall  f1-score   support

     Adenocarcinoma       0.58      0.76      0.66       120
     Adgelcarcinoma       0.58      0.41      0.48        51
Squamosgelcarcinoma       1.00      1.00      1.00        54
          Noncancer       0.64      0.49      0.55        90

           accuracy                           0.67       315
          macro avg       0.70      0.66      0.67       315
       weighted avg       0.67      0.67      0.66       315

Sostanzialmente un incremento dell'accuracy del 6% (adesso a 67%) e dell'F1 score di 10 punti (adesso a 0.67)


# Loss pesata sulle classi piccole -> Più utile nella classificazione binaria dove le classi sono più sbilanciate

In [ ]:
# Estrai le etichette (interi) dal dataset di training
labels = [label for _, label in train_dataset.samples]

from sklearn.utils.class_weight import compute_class_weight
import numpy as np # numpy is also needed for np.unique

# Calcola i pesi bilanciati per ciascuna classe
class_weights = compute_class_weight(class_weight='balanced', classes=np.unique(labels), y=labels)
print("Class weights:", class_weights)

# Converti in tensor e porta su device (CPU o GPU)
class_weights_tensor = torch.tensor(class_weights, dtype=torch.float)

In [ ]:
# passo i pesi come parametro in maniera tale che durante il training gli errori sulle classi meno frequenti abbiano un peso maggiore e quindi il modello sarà incentivato a non "ignorare" quelle classi.

# criterion = torch.nn.CrossEntropyLoss(weight=class_weights_tensor)


# ----------- EARLY STOPPING ------------
# # Early stopping logic
# if val_loss < best_val_loss:
#     best_val_loss = val_loss
#     epochs_without_improvement = 0
#     best_model_state = model.state_dict()  # salva il miglior modello
# else:
#     epochs_without_improvement += 1
#     print(f"  [EarlyStopping] No improvement for {epochs_without_improvement} epoch(s)")

# if epochs_without_improvement >= patience:
#     print("Early stopping triggered. Restoring best model weights.")
#     model.load_state_dict(best_model_state)
#     break